In [ ]:
import os
from pathlib import Path
import sys

from dotenv import load_dotenv
import pandas as pd


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.utils.google_services import set_up_google_connection

from src.utils.data_extracting import (
    extract_text_from_pdf,
    get_acc_files_from_gdrive_folder,
    get_all_transactions,
    get_balance_of_account,
    move_file,
)
from src.utils.data_loading import update_google_sheet
from src.utils.data_transforming import add_new_row, extract_transaction_info

In [ ]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')

In [ ]:
client, service = set_up_google_connection(credentials_path)

load_dotenv()
SPREADSHEET_ID = os.getenv('SPREADSHEET_2024_ID')
TEMP_FOLDER_ID = os.getenv('TEMP_FOLDER_ID')

In [ ]:
acc_states = get_acc_files_from_gdrive_folder(TEMP_FOLDER_ID, service)

# Read file

In [ ]:
file_id = acc_states[0]['id']
text = extract_text_from_pdf(file_id, service=service)
lines = text.split('\n')
len(lines)

In [ ]:
acc_balance_old = get_balance_of_account(lines, 'alter Kontostand')
acc_balance_new = get_balance_of_account(lines, 'neuer Kontostand')
print(f'Old acc value, Value: {acc_balance_old[0]}€ - Line-Index: {acc_balance_old[1]}')
print(f'New acc value, Value: {acc_balance_new[0]}€ - Line-Index: {acc_balance_new[1]}')

all_transactions = get_all_transactions(lines, acc_balance_old[1], acc_balance_new[1])
print(f'Count of transactions: {len(all_transactions)}')

# Open correct gsheet

In [ ]:
year = acc_states[0]['name'].split('_')[1]
year

In [ ]:
gsheet_file = f'SPREADSHEET_{year}_ID'
SPREADSHEET_ID = os.getenv(gsheet_file)
print(f'Using spreadsheet ID: {SPREADSHEET_ID}')

In [ ]:
spreadsheet = client.open_by_key(SPREADSHEET_ID)

sheet_incomes = spreadsheet.worksheet('Einnahmen')
sheet_expenses = spreadsheet.worksheet('Ausgaben')

df_expenses = pd.DataFrame(sheet_expenses.get_all_values())
df_incomes = pd.DataFrame(sheet_incomes.get_all_values())

In [ ]:
df_expenses.columns = df_expenses.iloc[0]
df_expenses = df_expenses[1:].reset_index(drop=True)

df_incomes.columns = df_incomes.iloc[0]
df_incomes = df_incomes[1:].reset_index(drop=True)

gsheets = {'Expense': df_expenses, 'Income': df_incomes}

In [ ]:
gsheets['Income'].head()

In [ ]:
month_cols = [f'{i:02}' for i in range(1, 13)]

# Alles außer Ziffern und Komma/Punkt rauswerfen, dann zu Zahl
gsheets['Income'][month_cols] = (
    gsheets['Income'][month_cols]
    .replace(r'[^0-9,.-]', '', regex=True)  # Währungen, Leerzeichen, etc. entfernen
    .replace(',', '.', regex=True)  # Kommas durch Punkt ersetzen (falls Kommazahlen)
    .apply(pd.to_numeric, errors='coerce')  # endlich in float/int umwandeln
)

# Alles außer Ziffern und Komma/Punkt rauswerfen, dann zu Zahl
gsheets['Expense'][month_cols] = (
    gsheets['Expense'][month_cols]
    .replace(r'[^0-9,.-]', '', regex=True)  # Währungen, Leerzeichen, etc. entfernen
    .replace(',', '.', regex=True)  # Kommas durch Punkt ersetzen (falls Kommazahlen)
    .apply(pd.to_numeric, errors='coerce')  # endlich in float/int umwandeln
)

In [ ]:
for transaction_index in range(len(all_transactions)):
    transaction = all_transactions[transaction_index]

    transaction_type, df, name, transaction_value, month = extract_transaction_info(
        transaction, gsheets
    )

    add_new_row(
        df,
        name,
        month,
        transaction_value,
        transaction_type,
        gsheets,
        general_account=True,
    )

In [ ]:
update_google_sheet(sheet_expenses, gsheets['Expense'])
update_google_sheet(sheet_incomes, gsheets['Income'])
print('✅ All changes saved to Google Sheets!')

# Move File

In [ ]:
# Beispielaufruf
file_id = acc_states[0]['id']
old_folder_id = TEMP_FOLDER_ID
new_folder_id = os.getenv('REGULAR_FOLDER_ID')
print(f'Moving file {file_id} from {old_folder_id} to {new_folder_id}')

In [ ]:
move_file(file_id, old_folder_id, new_folder_id)